In [92]:
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import pandas as pd

In [93]:
def build_dim_users(df_users):
    df = df_users.copy()

    # Align IDs with bookings (201.. instead of 1..)
    df["user_id"] = df["id"] + 200

    dim_users = pd.DataFrame(
        {
            "user_id": df["user_id"],
            "name": df.get("name"),
            "username": df.get("username"),
            "email": df.get("email"),
            "phone": df.get("phone"),
            "website": df.get("website"),
            "city": df.get("address.city"),
            "company": df.get("company.name"),
        }
    )

    # Drop potential duplicates on user_id to be safe
    dim_users = dim_users.drop_duplicates(subset=["user_id"])

    return dim_users

In [94]:
def build_dim_grounds(df_grounds):

    df = df_grounds.copy()

    dim_grounds = pd.DataFrame(
        {
            "ground_id": df["ground_id"],
            "ground_name": df["ground_name"],
            "location": df["city"],
            "ground_type": df["location_type"],
            "price_per_hour": df["price_per_hour"],
            "is_active": True,
        }
    )

    dim_grounds = dim_grounds.drop_duplicates(subset=["ground_id"])

    return dim_grounds

In [95]:
def build_dim_date(df_bookings):

    dates = pd.to_datetime(df_bookings["booking_date"], errors="coerce").dropna().drop_duplicates()
    dates = dates.sort_values()

    dim_date = pd.DataFrame({"full_date": dates})

    dim_date["date_id"] = range(1, len(dim_date) + 1)
    dim_date["day"] = dim_date["full_date"].dt.day
    dim_date["month"] = dim_date["full_date"].dt.month
    dim_date["year"] = dim_date["full_date"].dt.year
    dim_date["quarter"] = dim_date["full_date"].dt.quarter
    dim_date["weekday"] = dim_date["full_date"].dt.weekday + 1  # 1=Monday
    dim_date["weekday_name"] = dim_date["full_date"].dt.day_name()

    # Reorder columns to match schema more closely
    dim_date = dim_date[
        [
            "date_id",
            "full_date",
            "day",
            "month",
            "year",
            "quarter",
            "weekday",
            "weekday_name",
        ]
    ]

    return dim_date

In [96]:
def build_fact_bookings(df_bookings,dim_date):
 
    df = df_bookings.copy()

    # Ensure booking_date is datetime for joining
    df["booking_date"] = pd.to_datetime(df["booking_date"], errors="coerce")

    # Join to get date_id from dim_date
    df = df.merge(dim_date[["date_id", "full_date"]], left_on="booking_date", right_on="full_date", how="left")

    fact_bookings = pd.DataFrame(
        {
            "booking_id": df["booking_id"],
            "user_id": df["user_id"],
            "ground_id": df["ground_id"],
            "date_id": df["date_id"],
            "booking_date": df["booking_date"].dt.date,
            "slot_time": "Unknown",
            "duration_hours": df["hours"],
            "total_price": df["total_price"],
            "booking_status": "Completed",
        }
    )

    return fact_bookings

In [97]:
#fill numerical data
def fill_numeric(df):
    num_cols = df.select_dtypes(include=np.number).columns
    for col in num_cols:
        df[col] = df[col].fillna(df[col].median())
    return df

In [98]:
#fill categorical data
def fill_categorical(df):
    cat_cols = df.select_dtypes(include=object).columns
    for col in cat_cols:
        df[col] = df[col].fillna(df[col].mode().iloc[0] if not df[col].mode().empty else 'Unknown')
    return df

In [99]:
#remove duplicates
def remove_duplicates(df):
    df = df.drop_duplicates()
    return df

In [100]:
def remove_index(df):
    df = df.reset_index(drop=True)
    return df

In [101]:
#standerized names
def standarize_names(df):
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
    return df

In [102]:
# fix date format
def fix_date_format(df):
    date_mask = df.columns.str.contains('date|Date|DATE', case=False, na=False)
    date_columns = df.columns[date_mask]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

In [103]:
# outlier detection and treatment
def detect_remove_outliers(df):
    num_cols= df.select_dtypes(include=np.number).columns
    for col in num_cols:
        Q1 = df[col].quantile(0.20)
        Q3 = df[col].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    print(f"Outliers in {col}: {len(outliers)}")
    return df

In [104]:
# noise reduction
def noise_reduction(df):
    
    return df

In [105]:
def clean_grounds(df_grounds):
    df_grounds = fill_numeric(df_grounds)
    df_grounds = fill_categorical(df_grounds)
    df_grounds = remove_duplicates(df_grounds)
    df_grounds = standarize_names(df_grounds)
    df_grounds = remove_index(df_grounds)
    df_grounds = fix_date_format(df_grounds)
    df_grounds = detect_remove_outliers(df_grounds)
    print("Clean Grounds:")
    return df_grounds

In [106]:
def clean_bookings(df_bookings):
    df_bookings = fill_numeric(df_bookings)
    df_bookings = fill_categorical(df_bookings)
    df_bookings = remove_duplicates(df_bookings)
    df_bookings = standarize_names(df_bookings)
    df_bookings = remove_index(df_bookings)
    df_bookings = fix_date_format(df_bookings)
    df_bookings = detect_remove_outliers(df_bookings)
    print("Clean Bookings:")
    return df_bookings

In [107]:
def clean_users(df_users):
    df_users = fill_numeric(df_users)
    df_users = fill_categorical(df_users)
    df_users = remove_duplicates(df_users)
    df_users = standarize_names(df_users)
    df_users = remove_index(df_users)
    df_users = fix_date_format(df_users)
    df_users = detect_remove_outliers(df_users)
    print("Clean Users:")
    return df_users

In [108]:
def transform_data(df_bookings,df_grounds,df_users):
    
    #clean bookings
    df_bookings = df_bookings.copy()
    df_bookings = clean_bookings(df_bookings)

    df_grounds = df_grounds.copy()
    df_grounds = clean_grounds(df_grounds)

    df_users = df_users.copy()
    df_users = clean_users(df_users)

    # Build dimensions table
    dim_users = build_dim_users(df_users)
    dim_grounds = build_dim_grounds(df_grounds)
    dim_date = build_dim_date(df_bookings)

    # Build fact table
    fact_bookings = build_fact_bookings(df_bookings, dim_date)
    print(" ======== TRANSFORM ALL DATAS ===========")

    return dim_users, dim_grounds, dim_date, fact_bookings

In [109]:
# Test - Import from etl_py module
import sys
from pathlib import Path

# Remove previously imported modules
if 'etl_py' in sys.modules:
    del sys.modules['etl_py']
if 'etl_py.extract' in sys.modules:
    del sys.modules['etl_py.extract']
if 'etl_py.transform' in sys.modules:
    del sys.modules['etl_py.transform']

# Add parent directory to path so we can import etl_py
sys.path.insert(0, str(Path.cwd().parent))

from etl_py.extract import extract_data
from etl_py.transform import transform_data as transform_data_module

b, g, u = extract_data()
d_users, d_grounds, d_date, f_bookings = transform_data_module(b, g, u)
print("dim_users:", d_users.shape)
print("dim_grounds:", d_grounds.shape)
print("dim_date:", d_date.shape)
print("fact_bookings:", f_bookings.shape)

dim_users: (10, 8)
dim_grounds: (5, 6)
dim_date: (4, 8)
fact_bookings: (8, 9)


In [110]:
d_users

,user_id,name,username,email,phone,website,city,company
0,201,Leanne Graham,Bret,Sincere@april.biz,1-770-736-8031 x56442,hildegard.org,Gwenborough,Romaguera-Crona
1,202,Ervin Howell,Antonette,Shanna@melissa.tv,010-692-6593 x09125,anastasia.net,Wisokyburgh,Deckow-Crist
2,203,Clementine Bauch,Samantha,Nathan@yesenia.net,1-463-123-4447,ramiro.info,McKenziehaven,Romaguera-Jacobson
3,204,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,493-170-9623 x156,kale.biz,South Elvis,Robel-Corkery
4,205,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,(254)954-1289,demarco.info,Roscoeview,Keebler LLC
5,206,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,1-477-935-8478 x6430,ola.org,South Christy,Considine-Lockman
6,207,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,210.067.6132,elvis.io,Howemouth,Johns Group
7,208,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,586.493.6943 x140,jacynthe.com,Aliyaview,Abernathy Group
8,209,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,(775)976-6794 x41206,conrad.com,Bartholomebury,Yost and Sons
9,210,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,024-648-3804,ambrose.net,Lebsackbury,Hoeger LLC


In [111]:
d_date

,date_id,full_date,day,month,year,quarter,weekday,weekday_name
0,1,2025-01-05,5,1,2025,1,7,Sunday
2,2,2025-01-06,6,1,2025,1,1,Monday
4,3,2025-01-07,7,1,2025,1,2,Tuesday
6,4,2025-01-08,8,1,2025,1,3,Wednesday


In [112]:
d_grounds

,ground_id,ground_name,location,ground_type,price_per_hour,is_active
0,301,Goal Arena,Kathmandu,indoor,1000.0,True
1,302,KickOff Hub,Lalitpur,outdoor,1200.0,True
2,303,Futsal Pro,Bhaktapur,indoor,1100.0,True
3,304,Champions Turf,Kathmandu,outdoor,1300.0,True
4,305,Urban Futsal,Pokhara,indoor,900.0,True


In [113]:
f_bookings

,booking_id,user_id,ground_id,date_id,booking_date,slot_time,duration_hours,total_price,booking_status
0,1001,201,301,1,2025-01-05,Unknown,2.0,2000.0,Completed
1,1002,202,302,1,2025-01-05,Unknown,1.0,1200.0,Completed
2,1003,203,301,2,2025-01-06,Unknown,3.0,3000.0,Completed
3,1004,204,303,2,2025-01-06,Unknown,2.0,2200.0,Completed
4,1005,201,302,3,2025-01-07,Unknown,1.0,1200.0,Completed
5,1006,205,304,3,2025-01-07,Unknown,2.0,2600.0,Completed
6,1007,206,301,4,2025-01-08,Unknown,2.0,2000.0,Completed
7,1008,202,303,4,2025-01-08,Unknown,1.0,1100.0,Completed
